## Capstone Project: Where to open my business?

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         953 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The following packages will be UPDATED:

  conda                                        4.6.4-py36_0 --> 4.6.7-py36_0
  geopy              conda-forge/linux-64::geopy-1.11.0-py~ --> conda-forge/noarch::geopy-1.18.1-py_0



geopy-1.18.1         

### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [67]:
CLIENT_ID = 'YK02RX5M22FKW2CP0VN1HHTXU1XTMEYXKR0QNDV35GAODUBP' # your Foursquare ID
CLIENT_SECRET = 'EQVQ0QB00WU1TZPQ1YHEDJOSV1YQ3D5DM3J2Q0DVFDBPSHB3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YK02RX5M22FKW2CP0VN1HHTXU1XTMEYXKR0QNDV35GAODUBP
CLIENT_SECRET:EQVQ0QB00WU1TZPQ1YHEDJOSV1YQ3D5DM3J2Q0DVFDBPSHB3


In [94]:
address = 'Santiago de Surco, Peru'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-12.1251049 -76.9819193843966


In [77]:
latitude = -12.1251049 
longitude = -76.9819193843966

#### Define URL

In [56]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=YK02RX5M22FKW2CP0VN1HHTXU1XTMEYXKR0QNDV35GAODUBP&client_secret=EQVQ0QB00WU1TZPQ1YHEDJOSV1YQ3D5DM3J2Q0DVFDBPSHB3&ll=-12.1251049,-76.9819193843966&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [57]:
import requests

In [59]:
results = requests.get(url).json()
'There are {} around Santiago de Surco.'.format(len(results['response']['groups'][0]['items']))

'There are 24 around Santiago de Surco.'

#### Get relevant part of JSON

In [60]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '55fafebe498e47386ac6fca9',
  'name': 'Dalcello',
  'location': {'address': 'Caminos del Inca 1502',
   'lat': -12.125442,
   'lng': -76.98328,
   'labeledLatLngs': [{'label': 'display',
     'lat': -12.125442,
     'lng': -76.98328}],
   'distance': 152,
   'cc': 'PE',
   'city': 'Surco',
   'state': 'Lima',
   'country': 'Perú',
   'formattedAddress': ['Caminos del Inca 1502', 'Surco', 'Perú']},
  'categories': [{'id': '4bf58dd8d48988d1c9941735',
    'name': 'Ice Cream Shop',
    'pluralName': 'Ice Cream Shops',
    'shortName': 'Ice Cream',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-55fafebe498e47386ac6fca9-0'}

#### Process JSON and convert it to a clean dataframe

In [61]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Dalcello,Ice Cream Shop,Caminos del Inca 1502,PE,Surco,Perú,NaN,152,"[Caminos del Inca 1502, Surco, Perú]","[{'label': 'display', 'lat': -12.125442, 'lng': -76.98328}]",-12.125442,-76.983280,NaN,NaN,Lima,55fafebe498e47386ac6fca9
1,Il Piccolo Pomodoro,Italian Restaurant,Av. Caminos del Inca 1473,PE,Surco,Perú,NaN,202,"[Av. Caminos del Inca 1473, Santiago de Surco, Perú]","[{'label': 'display', 'lat': -12.12463614317688, 'lng': -76.98371976845327}]",-12.124636,-76.983720,NaN,NaN,Lima,4d2520f05886a09316bcf052
2,Tierra Santa,Middle Eastern Restaurant,Av. Caminos del Inca 1512,PE,Surco,Perú,NaN,142,"[Av. Caminos del Inca 1512, Santiago de Surco, 33, Perú]","[{'label': 'display', 'lat': -12.125326565832657, 'lng': -76.98320871301907}]",-12.125327,-76.983209,NaN,33,Lima,4cd4382308f48cfa9141a585
3,La Verdad de la Milanesa,Restaurant,Av. Caminos del Inca 1478,PE,Surco,Perú,NaN,212,"[Av. Caminos del Inca 1478, Surco, LIMA 33, Perú]","[{'label': 'display', 'lat': -12.124947435315415, 'lng': -76.98386382965161}]",-12.124947,-76.983864,NaN,LIMA 33,Lima,58eaa13ecc5b6a79746550dc
4,Roll-Star Caminos Del Inca,Sushi Restaurant,Jiron Batallon Callao Sur 187,PE,Santiago de Surco,Perú,NaN,189,"[Jiron Batallon Callao Sur 187, Santiago de Surco, 15039, Perú]","[{'label': 'display', 'lat': -12.125381554041354, 'lng': -76.98363576817897}]",-12.125382,-76.983636,NaN,15039,Municipalidad Metropolitana de Lima,59cfeee7851de5715718b138
5,Parque Juan Bielovuciv (Las Ardillas),Park,Ca. Narciso de la Colina 100,PE,Surco,Perú,Ca. Enrique Campos,366,"[Ca. Narciso de la Colina 100 (Ca. Enrique Campos), Surco, Perú]","[{'label': 'display', 'lat': -12.125560028962742, 'lng': -76.985258021448}]",-12.125560,-76.985258,NaN,NaN,Lima,4d28d6238292236a02491dbb
6,Chifa San Joy Lao,Chinese Restaurant,Av. Caminos del Inca 1893,PE,Surco,Perú,esq. Alguaciles,441,"[Av. Caminos del Inca 1893 (esq. Alguaciles), Santiago de Surco, 33, Perú]","[{'label': 'display', 'lat': -12.129051376972207, 'lng': -76.98151436045495}]",-12.129051,-76.981514,NaN,33,Lima,4b9d4127f964a5206a9d36e3
7,Don Mamino,Bakery,Av. Velasco Astete 1621,PE,Surco,Perú,NaN,446,"[Av. Velasco Astete 1621, Santiago de Surco, 33, Perú]","[{'label': 'display', 'lat': -12.123240882803039, 'lng': -76.98555520061755}]",-12.123241,-76.985555,NaN,33,Lima,4bbbc4bc51b89c7413af862a
8,Chiwake - comida peruana,Restaurant,Av. Caminos Del Inca 1533,PE,Surco,Perú,NaN,113,"[Av. Caminos Del Inca 1533, Surco, Perú]","[{'label': 'display', 'lat': -12.125389373223669, 'lng': -76.98291877658987}]",-12.125389,-76.982919,NaN,NaN,Lima,563e4520cd10e09675e9fb76
9,Chifa Fu Sen,Chinese Restaurant,Av. Caminos del Inca 1515-a,PE,Surco,Perú,Casi esq. Con Batallon Callao,137,"[Av. Caminos del Inca 1515-a (Casi esq. Con Batallon Callao), Santiago de Surco, 33, Perú]","[{'label': 'display', 'lat': -12.125263968382857, 'lng': -76.98317439189661}]",-12.125264,-76.983174,NaN,33,Lima,4d584c1c7c0b370463f3e008


#### Let's visualize these items on the map around our location

In [62]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [97]:
data_fil = dataframe_filtered[["name","categories", "lat","lng"]]

### Clustering: Using Kmeans

In [119]:
data_fil.head()

,name,categories,lat,lng
0,Dalcello,Ice Cream Shop,-12.125442,-76.983280
1,Il Piccolo Pomodoro,Italian Restaurant,-12.124636,-76.983720
2,Tierra Santa,Middle Eastern Restaurant,-12.125327,-76.983209
3,La Verdad de la Milanesa,Restaurant,-12.124947,-76.983864
4,Roll-Star Caminos Del Inca,Sushi Restaurant,-12.125382,-76.983636


In [111]:
map_santiago = folium.Map(location=[latitude, longitude], zoom_start=100)
map_santiago

In [145]:
# add markers to map
for lat, lng, borough, neighborhood in zip(data_fil['lat'], data_fil['lng'], data_fil['name'], data_fil['categories']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_santiago)  
    
map_santiago

In [118]:
from sklearn.cluster import KMeans

In [123]:
df_join_clustering = data_fil.drop('name', 1)
df_join_clustering = df_join_clustering.drop('categories', 1)
df_join_clustering["Cluster"] = 0
df_join_clustering["Cluster Labels"] = 0

In [126]:
df_join_clustering = df_join_clustering[["Cluster","Cluster Labels", "lat", "lng"]]
df_join_clustering.head()

,Cluster,Cluster Labels,lat,lng
0,0,0,-12.125442,-76.983280
1,0,0,-12.124636,-76.983720
2,0,0,-12.125327,-76.983209
3,0,0,-12.124947,-76.983864
4,0,0,-12.125382,-76.983636


In [129]:
# per the pattern when looking at new york, let's try 5 clusters with toronto
kclusters = 3

# k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_join_clustering)


In [130]:
df_resx = data_fil
df_resx.insert(0, 'Cluster Label', kmeans.labels_)
df_resx

,Cluster Label,name,categories,lat,lng
0,2,Dalcello,Ice Cream Shop,-12.125442,-76.983280
1,2,Il Piccolo Pomodoro,Italian Restaurant,-12.124636,-76.983720
2,2,Tierra Santa,Middle Eastern Restaurant,-12.125327,-76.983209
3,2,La Verdad de la Milanesa,Restaurant,-12.124947,-76.983864
4,2,Roll-Star Caminos Del Inca,Sushi Restaurant,-12.125382,-76.983636
5,2,Parque Juan Bielovuciv (Las Ardillas),Park,-12.125560,-76.985258
6,1,Chifa San Joy Lao,Chinese Restaurant,-12.129051,-76.981514
7,2,Don Mamino,Bakery,-12.123241,-76.985555
8,2,Chiwake - comida peruana,Restaurant,-12.125389,-76.982919
9,2,Chifa Fu Sen,Chinese Restaurant,-12.125264,-76.983174


In [133]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [136]:
df_resx

,Cluster Label,name,categories,lat,lng
0,2,Dalcello,Ice Cream Shop,-12.125442,-76.983280
1,2,Il Piccolo Pomodoro,Italian Restaurant,-12.124636,-76.983720
2,2,Tierra Santa,Middle Eastern Restaurant,-12.125327,-76.983209
3,2,La Verdad de la Milanesa,Restaurant,-12.124947,-76.983864
4,2,Roll-Star Caminos Del Inca,Sushi Restaurant,-12.125382,-76.983636
5,2,Parque Juan Bielovuciv (Las Ardillas),Park,-12.125560,-76.985258
6,1,Chifa San Joy Lao,Chinese Restaurant,-12.129051,-76.981514
7,2,Don Mamino,Bakery,-12.123241,-76.985555
8,2,Chiwake - comida peruana,Restaurant,-12.125389,-76.982919
9,2,Chifa Fu Sen,Chinese Restaurant,-12.125264,-76.983174


In [142]:
# df_resx
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_resx['lat'], df_resx['lng'], df_resx['name'], df_resx['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
